In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [32]:
import json
import csv
import re
import torch
import pandas as pd
import time
import operator
import math
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torch.optim import AdamW
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import BertModel, BertTokenizer, BertConfig, BertPreTrainedModel, AdamW
from sklearn import preprocessing
from transformers.data.processors.utils import InputExample, DataProcessor

In [33]:
DEV_FILE = '/kaggle/input/preprocessed-data/dev.csv'
TRAIN_FILE = '/kaggle/input/preprocessed-data/train.csv'
TEST_FILE = '/kaggle/input/preprocessed-data/test.csv'

# over-sampling non-rumour data
train_set = pd.read_csv(TRAIN_FILE)
train_set_rumour_idx = train_set.index[train_set['Label'] == 'rumour'].tolist()
ratio = math.floor(train_set['Label'].value_counts()[0]/train_set['Label'].value_counts()[1])
while ratio:
    train_set = pd.concat([train_set, train_set.loc[train_set_rumour_idx]], ignore_index=True, axis=0)
    ratio -= 1
train_set.to_csv('./train.csv', index = False)
TRAIN_FILE = '/kaggle/working/train.csv'

dev_set = pd.read_csv(DEV_FILE)
dev_set_rumour_idx = dev_set.index[dev_set['Label'] == 'rumour'].tolist()
ratio = math.floor(dev_set['Label'].value_counts()[0]/dev_set['Label'].value_counts()[1])
while ratio:
    dev_set = pd.concat([dev_set, dev_set.loc[dev_set_rumour_idx]], ignore_index=True, axis=0)
    ratio -= 1
dev_set.to_csv('./dev.csv', index = False)
DEV_FILE = '/kaggle/working/dev.csv'

In [34]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(['nonrumour', 'rumour']) # converts 'nonrumour' to 0 and 'rumour' to 1

train_events = pd.read_csv(TRAIN_FILE)
train_events['Label'] = label_encoder.transform(train_events['Label'])
dev_events = pd.read_csv(DEV_FILE)
dev_events['Label'] = label_encoder.transform(dev_events['Label'])

In [35]:
def text_split(text):
    #text = text.replace('[CLS]','')
    #text = text.replace('[SEP]','')
    new_text = ''
    curr_text = ''
    
    for word in text.split():
        curr_text += word + ' '
        
        if word == '[SEP]':
            new_text += curr_text
            curr_text = ''
            
    new_text = new_text[:-1]
    
    return new_text

split_train_events = train_events.copy()
split_train_events['Event Tweets'] = split_train_events['Event Tweets'].apply(text_split)
split_dev_events = dev_events.copy()
split_dev_events['Event Tweets'] = split_dev_events['Event Tweets'].apply(text_split)

In [36]:
train_max = 0
curr_count = 0

for i in split_train_events['Event Tweets']:
    for j in i.split():
        curr_count += 1
        if j == '[SEP]':
            if curr_count > train_max:
                train_max = curr_count
            curr_count = 0
        
dev_max = 0
curr_count = 0

for i in split_dev_events['Event Tweets']:
    for j in i.split():
        curr_count += 1
        if j == '[SEP]':
            if curr_count > dev_max:
                dev_max = curr_count
            curr_count = 0
        
# max counts used for padding later on
print("Training set has a max sentence length of", train_max)
print("Development set has a max sentence length of", dev_max)

In [37]:
class BertClassifier(nn.Module):

    def __init__(self, bert_base):
        super(BertClassifier, self).__init__()
        self.bert = bert_base
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768,512)
        self.fc2 = nn.Linear(512, 2)
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, input_ids, attention_mask):
        
        _, cls_rep = self.bert(input_ids, attention_mask = attention_mask, return_dict = False)
        
        x = self.fc1(cls_rep)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        
        return x

In [38]:
bert_base = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

train_tokens = tokenizer.batch_encode_plus(
    list(split_train_events['Event Tweets']),
    add_special_tokens = False,
    max_length = train_max,
    pad_to_max_length = True,
    truncation = True
)

dev_tokens = tokenizer.batch_encode_plus(
    list(split_dev_events['Event Tweets']),
    add_special_tokens = False,
    max_length = dev_max,
    pad_to_max_length = True,
    truncation = True
)

In [39]:
model = BertClassifier(bert_base)
model.to('cuda')

In [40]:
opti = AdamW(model.parameters(), lr = 2e-5)
criterion = nn.NLLLoss()
max_epochs = 10
batch_size = 3

In [41]:
train_ids = torch.tensor(train_tokens['input_ids'])
train_mask = torch.tensor(train_tokens['attention_mask'])
train_y = torch.tensor((list(split_train_events['Label'])))
train_data = TensorDataset(train_ids, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)

dev_ids = torch.tensor(dev_tokens['input_ids'])
dev_mask = torch.tensor(dev_tokens['attention_mask'])
dev_y = torch.tensor(list(split_dev_events['Label']))
dev_data = TensorDataset(dev_ids, dev_mask, dev_y)
dev_sampler = RandomSampler(dev_data)
dev_loader = DataLoader(dev_data, sampler = dev_sampler, batch_size = batch_size)

In [42]:
def train():
    
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    all_preds = []
    
    for step, (ids, masks, labels) in enumerate(train_loader):            
        ids, masks, labels = ids.to('cuda'), masks.to('cuda'), labels.to('cuda')
        
        model.zero_grad()

        preds = model(ids, masks)

        loss = criterion(preds, labels)
        total_loss = total_loss + loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opti.step()

        preds = preds.detach().cpu().numpy()
        all_preds.append(preds)

    avg_loss = total_loss / len(train_loader)
    all_preds = np.concatenate(all_preds, axis = 0)
    
    return avg_loss, all_preds

In [43]:
def evaluate():
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    all_preds = []
    
    for step, (ids, masks, labels) in enumerate(dev_loader):
        ids, masks, labels = ids.to('cuda'), masks.to('cuda'), labels.to('cuda')

        with torch.no_grad():
            preds = model(ids, masks)
            
            loss = criterion(preds, labels)
            total_loss = total_loss + loss.item()
            
            preds = preds.detach().cpu().numpy()
            all_preds.append(preds)
    
    avg_loss = total_loss / len(dev_loader)
    all_preds = np.concatenate(all_preds, axis = 0)
    
    return avg_loss, all_preds

In [44]:
best_dev_loss = float('inf')

train_loses = []
dev_loses = []

for ep in range(max_epochs):
    print("Epoch " + str(ep + 1) + ":")
    
    train_loss, _ = train()
    dev_loss, _ = evaluate()
    
    if dev_loss < best_dev_loss:
        best_dev_loss = dev_loss
        torch.save(model.state_dict(), 'best_weights.pt')
        
    train_loses.append(train_loss)
    dev_loses.append(dev_loss)
    
    print(f"\nTraining Loss: {train_loss:.3f}")
    print(f"\nDevelopment Loss: {dev_loss:.3f}")

In [49]:
test_events = pd.read_csv(TEST_FILE)
split_test_events = test_events.copy()
split_test_events['Event Tweets'] = split_test_events['Event Tweets'].apply(text_split)

test_max = 0
curr_count = 0

for i in split_test_events['Event Tweets']:
    for j in i.split():
        curr_count += 1
        if j == '[SEP]':
            if curr_count > test_max:
                test_max = curr_count
            curr_count = 0
            
print("Test set has a max sentence length of", test_max)

In [50]:
test_tokens = tokenizer.batch_encode_plus(
    list(split_test_events['Event Tweets']),
    add_special_tokens = False,
    max_length = train_max,
    pad_to_max_length = True,
    truncation = True
)

test_ids = torch.tensor(test_tokens['input_ids'])
test_mask = torch.tensor(test_tokens['attention_mask'])

test_data = TensorDataset(test_ids, test_mask)
test_sampler = RandomSampler(test_data)
test_loader = DataLoader(test_data, sampler = test_sampler, batch_size = batch_size)

In [60]:
weight_file = f'best_weights.pt'
model.load_state_dict(torch.load(weight_file))

with torch.no_grad():
    preds = model(test_ids.to('cuda'), test_mask.to('cuda'))
    preds = preds.detach().cpu().numpy()
    preds = np.argmax(preds, axis = 1)

In [61]:
with open('/kaggle/working/final.csv', 'w', newline='') as o:
    csv_write = csv.writer(o)
    csv_write.writerow(['Id', 'Predicted'])

    for i in range(len(preds)):
        idx = i
        is_rumour = preds[i]
        csv_write.writerow([idx, is_rumour])